# Implementacion de el metodo de la maxima verosimilitud para la regresion logistica

## Definir la funcion de entorno L(b)

In [1]:
from IPython.display import display, Math, Latex

$ L(\beta)=\sum_{i=1}^n P_i^{y_i}(1-Pi)^{y_i}$

In [2]:
def likelihood(y,pi):
    import numpy as np
    total_sum = 1
    sum_in = list(range(1,len(y)+1))
    for i in range(len(y)):
        sum_in[i] = np.where(y[i]==1, pi[i], 1-pi[i])
        total_sum = total_sum * sum_in[i]
    return total_sum

## Calcular las probabilidades para cada observacion

$ Pi = P(x_i) = \frac{1}{1+e^{-\sum{j=0}^k\cdot x_{ij}}}$

In [3]:
def logitprobs(X,beta):
    import numpy as np
    n_rows = np.shape(X)[0]
    n_cols = np.shape(X)[1]
    pi=list(range(1,n_rows+1))
    expon=list(range(1,n_rows+1))
    for i in range(n_rows):
        expon[i]=0
        for j in range(n_cols):
            ex=X[i,j]*beta[j]
            expon[i] = ex + expon[i]
        with np.errstate(divide="ignore", invalid="ignore"):
            pi[i]=1/(1+np.exp(-expon[i]))
    return pi

## Calcular la matriz diagonal w

$ W= diag(P_i \cdot (1-P_i))_{i=1}^n$

In [4]:
def findW(pi):
    import numpy as np
    n = len(pi)
    W = np.zeros(n*n).reshape(n,n)
    for i in range(n):
        print(i)
        W[i,i]=pi[i]*(1-pi[i])
        W[i,i].astype(float)
    return W

## Obtener la solucion de la funcion logistica

$ \beta{n+1} = \beta_n -\frac{f(\beta_n)}{f'(\beta_n)}$

$f(\beta) = X(Y-P)$

$f'(\beta) = XWX^T$

In [5]:
def logistics(X, Y, limit):
    import numpy as np
    from numpy import linalg
    nrow = np.shape(X)[0]
    bias = np.ones(nrow).reshape(nrow,1)
    X_new = np.append(X, bias, axis=1)
    ncol = np.shape(X_new)[1]
    beta = np.zeros(ncol).reshape(ncol,1)
    root_dif = np.array(range(1,ncol+1)).reshape(ncol,1)
    iter_i = 10000
    while(iter_i>limit):
        print("Iter:i"+str(iter_i)+",limit:"+str(limit))
        pi = logitprobs(X_new, beta)
        print("Pi:"+str(pi))
        W = findW(pi)
        print("W:"+str(W))
        num = (np.transpose(np.matrix(X_new))*np.matrix(Y - np.transpose(pi)).transpose())
        den = (np.matrix(np.transpose(X_new))*np.matrix(W)*np.matrix(X_new))
        root_dif = np.array(linalg.inv(den)*num)
        beta = beta + root_dif
        print("Beta:"+str(beta))
        iter_i = np.sum(root_dif*root_dif)
        print("Iteri:"+str(iter_i))
        l1 = likelihood(Y, pi)
    return beta       

## Comprobacion experimental

In [6]:
import numpy as np

In [7]:
X = np.array(range(10)).reshape(10,1)

In [8]:
X

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [9]:
Y = [0,0,0,0,1,0,1,0,1,1]

In [10]:
bias = np.ones(10).reshape(10,1)
X_new = np.append(X, bias, axis=1)

In [11]:
X_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

In [12]:
a = logistics(X,Y,0.00001)

Iter:i10000,limit:1e-05
Pi:[array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
0
1
2
3
4
5
6
7
8
9
W:[[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25]]
Beta:[[ 0.43636364]
 [-2.36363636]]
Iteri:5.777190082644626
Iter:i5.777190082644626,limit:1e-05
Pi:[array([0.08598797]), array([0.12705276]), array([0.18378532]), array([0.2583532]), array([0.35019508]), array([0.45467026]), array([0.56329497]), array([0.66616913]), array([0.755

In [13]:
l1 = likelihood(Y, logitprobs(X,a))

In [14]:
l1

array([1.32622426e-06])

In [15]:
Y = 0.66220827 * X - 3.69557172

## Con el paquete stats de Python

In [17]:
import statsmodels.api as sm

In [19]:
logit_model = sm.Logit(Y,X_new)
result = logit_model.fit()

ValueError: endog must be in the unit interval.